In [1]:
import sys
sys.path.append('..')
# reload local packages automatically
%load_ext autoreload
%autoreload 2

from jax import config
config.update("jax_enable_x64", True)

# Import packages.
import cvxpy as cp
import numpy as np

import matplotlib.pyplot as plt


from opentn.transformations import create_kitaev_liouvillians, exp_operator_dt, factorize_psd, super2choi
d, N = 2, 4
dim = d**N
Lvec, Lvec_odd, Lvec_even, Lnn = create_kitaev_liouvillians(N=N, d=d, gamma=1e-2)
tau = 4
superops_exp = []
for i, op in enumerate([Lvec, Lvec_odd, Lvec_even]):
    if i == 1:
        superops_exp.append(exp_operator_dt(op, tau/2, 'jax'))
    else:
        superops_exp.append(exp_operator_dt(op, tau, 'jax'))
exp_Lvec, exp_Lvec_odd, exp_Lvec_even = superops_exp

X1 = factorize_psd(psd=super2choi(exp_Lvec_odd), tol=1e-12)
X2 = factorize_psd(psd=super2choi(exp_Lvec_even), tol=1e-12)

from opentn.optimization import gds, frobenius_norm, model_Ys, compute_loss
cost_n4, grads_n4, xi_n4 = gds(fn=compute_loss, x0=[X1, X2, X1], iter=int(2e3), loss_fn=frobenius_norm, model=model_Ys, rate=1.5e-9, exact = exp_Lvec, show_cost=False)
print(cost_n4[0])
print(cost_n4[-1])

2.6383007149418305e-07
1.4977787039695335e-07


In [2]:
from opentn.transformations import super2choi, choi_composition
from opentn.optimization import small2zero
from scipy import sparse

# rest of matrices optimized with GSD
X1_gds, X2_gds, X3_gds = xi_n4[-1]
Y1_gds = X1_gds@X1_gds.conj().T
Y2_gds = X2_gds@X2_gds.conj().T
Y3_gds = X3_gds@X3_gds.conj().T

tol = 1e-10

# rhs to which we want to approximate
rhs = np.asarray(super2choi(superop=exp_Lvec, dim=dim))
# srhs = sparse.csr_matrix(rhs) # small2zero was deleted
srhs = sparse.csr_matrix(small2zero(rhs, tol=tol)) # small2zero was deleted
srhs = srhs.astype(np.float64)

/opt/miniconda3/envs/thesis/lib/python3.10/site-packages/scipy/sparse/_data.py:72: ComplexWarning: Casting complex values to real discards the imaginary part
  self.data.astype(dtype, casting=casting, copy=True),


In [5]:
from opentn.transformations import link_product_cvxpy
Y1_cvxpy = cp.Variable((dim**2,dim**2), PSD=True)
C2 = choi_composition(small2zero(Y2_gds, tol), small2zero(Y3_gds, tol))
lhs = link_product_cvxpy(C1=Y1_cvxpy, C2=C2, dim=dim, transpose=1, optimization=True)
cost = cp.norm(lhs - srhs, "fro")
prob = cp.Problem(cp.Minimize(cost))
Y1_cvxpy.value = small2zero(Y1_gds, tol=tol).astype(np.float64) 
cost.value

/opt/miniconda3/envs/thesis/lib/python3.10/site-packages/scipy/sparse/_data.py:72: ComplexWarning: Casting complex values to real discards the imaginary part
  self.data.astype(dtype, casting=casting, copy=True),
/var/folders/lv/986s3mp97gv9n3rb757bg87m0000gn/T/ipykernel_9055/826646750.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  Y1_cvxpy.value = small2zero(Y1_gds, tol=tol).astype(np.float64)


1.498151455427664e-07

In [6]:
prob.solve(solver=cp.SCS, verbose=True, canon_backend=cp.SCIPY_CANON_BACKEND, eps=tol)

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jul 27 02:58:17 PM: Your problem has 65536 variables, 0 constraints, and 0 parameters.
(CVXPY) Jul 27 02:58:17 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 27 02:58:17 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 27 02:58:17 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 27 02:58:17 PM: Compiling problem (target solver=SCS).
(CVXPY) Jul 27 02:58:17 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

9.517953777109907e-08

In [12]:
# checking if the solution with and without approximation coincide
# 1. all have small2zero except solution. (should coincide with the output of solver)
print(frobenius_norm(choi_composition(Y1_cvxpy.value, C2), small2zero(rhs, tol=tol)))
# 2. all have small2zero including solution. Due to approximation here, the value increases a bit
print(frobenius_norm(choi_composition(small2zero(Y1_cvxpy.value, tol=tol), C2), small2zero(rhs, tol=tol)))
# 3. none of them have small2zero
print(frobenius_norm(choi_composition(Y1_cvxpy.value, choi_composition(Y2_gds, Y3_gds)), rhs))

9.517953776939443e-08
9.518249209451488e-08
9.521364641951719e-08


In [15]:
# verying the rank of: initial_guess, gds_sol, cvx solution w/and without small2zero
max_idx = 20
for op in [X1, Y1_gds, Y1_cvxpy.value, small2zero(Y1_cvxpy.value, tol=tol)]:
    print(np.linalg.matrix_rank(op, tol=tol))
    print(np.sort(np.linalg.svd(op, compute_uv=False))[::-1][:max_idx])
    print('----')


4
[3.99501248e+00 1.41156678e-01 1.41156678e-01 4.98752081e-03
 1.68086221e-15 1.54951812e-15 1.30661439e-15 1.16206956e-15
 1.04950910e-15 9.51159912e-16 9.49231710e-16 8.98450268e-16
 8.07706366e-16 7.89423078e-16 6.88379074e-16 6.77265483e-16
 6.22736767e-16 6.08753246e-16 5.78770075e-16 5.48590551e-16]
----
4
[1.59601248e+01 1.99251812e-02 1.99251787e-02 2.48753821e-05
 7.40452593e-15 6.23416821e-15 5.78733966e-15 5.45129161e-15
 5.02854100e-15 4.95731989e-15 4.56542003e-15 4.34589312e-15
 4.19879763e-15 4.02361484e-15 3.80645554e-15 3.57183712e-15
 3.04235402e-15 2.81870941e-15 2.70461589e-15 2.24288391e-15]
----
16
[1.59601248e+01 1.99251773e-02 1.99251750e-02 2.48854571e-05
 5.53564209e-08 4.75268505e-08 4.47013493e-08 4.04359112e-08
 3.85099712e-08 1.87179267e-08 1.34944648e-08 8.18230425e-09
 2.93693059e-10 2.37246938e-10 1.88128013e-10 1.09482978e-10
 8.39013459e-11 7.73763815e-11 6.18413108e-11 6.15472486e-11]
----
27
[1.59601248e+01 1.99251773e-02 1.99251750e-02 2.48854571e